## 미등록단어 문제 (out of vocabulary)

몇 개의 문장에 대해서 학습된 모델 기반으로 작동하는 형태소 분석기를 이용하여 명사를 추출하여 봅시다.

이 튜토리얼을 만들 때의 KoNLPy의 버전은 0.5.1입니다.

In [1]:
# -*- encoding:utf8 -*-
import sys
if sys.version_info <= (2,7):
    reload(sys)
    sys.setdefaultencoding('utf-8')
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

kkma = Kkma()
twitter = Okt()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.1


'박근혜'라는 단어는 여러 정치 관련 문서에서 등장하였기 때문에 사전에 등장할 가능성이 높음에도, 꼬꼬마의 경우 박근혜를 '박근 + 혜'와 '박근혜' 중 어떤 것이 명사인지 혼동합니다. '최순실'의 경우 학습데이터에 이 단어가 들어갈리가 거의 없기 때문에 대부분 형태소분석기들이 '최순실'이라는 명사를 인식하지 못합니다. 

In [2]:
sent = '박근혜 최순실의 국정농단 사태는 대통령탄핵으로 이어졌습니다'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('트위터 명사: ', twitter.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['박근', '박근혜', '혜', '최', '최순실의', '순', '실의', '국정', '국정농단', '농단', '사태', '대통령', '대통령탄핵', '탄핵']
트위터 명사:  ['박근혜', '최', '실의', '국정', '농단', '사태', '대통령', '탄핵']
한나눔 명사:  ['박근혜', '최순실', '국정농단', '사태', '대통령탄핵', '이']


위와 비슷하게 '트와이스', '아이오아이'이 시간이 지나면서 발생하는 단어들은 명사로 인식하기가 매우 힘듭니다. 모르는 어절이 입력되면 이를 명사로 판단하는 휴리스틱들을 대부분 이용합니다. 사실 많은 양의 단어들은 명사이기 때문에 이는 매우 합리적인 fallback입니다. 트와이스 노래 'tt'의 경우에는 심지어 외국어이기 때문에 애초에 외국어로 분리가 되어 명사로 인식되지 못합니다. 

In [3]:
sent = '트와이스 그리고 아이오아이 좋아여 tt가 저번에 1위 했었죠?'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('트위터 명사: ', twitter.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['트와', '트와이스', '이스', '아이오', '아이오아이', '아이', '여', '저번', '1', '1위', '위']
트위터 명사:  ['트와이스', '아이오', '아이', '저번', '위']
한나눔 명사:  ['트와이스', '아이오아', '좋아', '저번', '1위']


'보코하람'과 같이 외국어가 들어오면 이를 분해하는 특징도 있습니다. '보코하람'은 단어로 알지 못하지만 '보', '코' 라는 것은 명사로 알고 있기 때문입니다. 

In [4]:
sent = '보코하람 테러로 소말리아에서 전쟁이 있었어요'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('트위터 명사: ', twitter.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['보', '보코', '코', '테러', '소말리', '전쟁']
트위터 명사:  ['보코하람', '테러', '소말리아', '전쟁']
한나눔 명사:  ['보코하람', '테러', '소말리아', '전쟁']


## L-R 구조를 이용한 명사 추출

하지만 이는 학습데이터의 관점에서 인식하기 어려운 명사이지, 매일 뉴스를 보는 우리는 모두 인식할 수 있는 단어들입니다. 

    오늘 A에서 저녁먹자
    너랑간 A 진짜 좋더라
    기다릴테니까 곧바로 A로 와
    난 A가 좋더라
    
위 세 문장을 보면 A라는 단어는 명사라는 것을 우리는 쉽게 눈치챕니다. 왜냐면 A 다음에 오는 단어들이 [-에서, -로, -가]와 같이 명사 다음에 오는 조사들이 자주 등장하기 때문입니다. 

한국어는 L + [R] 구조입니다. (이 내용은 [wordextractor_lecture.ipynb][wordextractor_lecture] 를 보고 오시기 바랍니다.) L 옆에 등장하는 R의 분포는 L이 명사인지 아닌지를 판단하는 좋은 힌트가 됩니다. 하지만 조사들을 R로 가지고 있으면서 규칙기반으로 명사를 찾을 수는 없습니다. '-은'은 대표적인 조사입니다. 하지만 '손나은'이 '손나 + 은'은 아닙니다. 연예 뉴스에서 '에이핑크 맴버 손나은'은 자주 등장하지만 '손나 + 은', '손나 + 이', '손나 + 에게' 같은 어절은 자주 등장하지 않습니다. 

우리는 이와 같이 L - R의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단합니다. 세종 말뭉치를 이용하여 명사 뒤에 등장하는 R set을 모아뒀으며, R set의 단어들은 명사 가능 점수가 학습되어 있습니다. 명사 가능 점수의 범위는 [-1, 1]입니다. 

    내서	   -0.530702
    있게	    1.000000
    있는	    0.327824
    쓰는	    0.079298
    었다며	   -1.000000
    였다며	    0.437399
    
'했 + 었다며'이라면 '했'은 명사 점수가 -1.0입니다. '재미 + 있게' 3번, '재미 + 있는' 2번 등장하였다면 재미의 명사 가능 점수는 (3 x 1.0 + 2 x 0.33) / 5 = 0.732점입니다. 명사 추출의 threshold가 0.5라면 '재미'는 명사로 추출됩니다. 

이 방법은 주어진 문서집합에서 어절들의 구조를 학습하여 그 주어진 문서집합의 명사를 추출합니다. 학습데이터가 필요하지 않은 통계 기반의 unsupervised 학습방법입니다. 

[wordextractor_lecture]: wordextractor_lecture.ipynb

## LRNounExtractor

후처리 과정과 스코어링 과정을 여러 가지 시도해 보는 중입니다. 현재 버전 (0.0.4) 에서는 두 가지의 명사 추출기를 제공합니다만, 이후 명사 추출기들이 하나로 통합될 수 있습니다. 

LRNounExtractor 는 가장 가볍게 만든 명사 추출기입니다. 

In [5]:
import sys
sys.path.insert(0, '../')
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
All 2398 r features was loaded


NounExtractor.train(sents)의 sents는 len(sents)를 할 수 있는 list-like이면 됩니다. DoublespaceLineCorpus 는 한 문장이 하나의 문서이며, 한 문서 내의 문장 구분을 두 칸 띄어쓰기 형식으로 저장한 텍스트 형식입니다. iter_sent=True 이면 문서가 아닌 문장 단위로 yield 를 수행합니다. 아래의 클래스는 다음처럼 import 할 수 있습니다. 

    from soynlp.utils import DoublespaceLineCorpus

튜토리얼에 이용한 문서는 2016년 10월 20일의 뉴스로부터 한글로 이뤄진 223,357 문장입니다. 

In [6]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = 'YOUR_DoubleSpaceLine_Corpus'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

223357

추출하고 싶은 명사의 noun score threshold와 명사의 최소빈도수 (min count)를 parameter로 넣으면 명사 점수 0.3점 이상, 문서집합에서 등장한 빈도가 100번 이상인 단어들에서 명사를 추출합니다. 

train 함수에서는 WordExtractor를 통하여 단어를 미리 추출하고, 이들 중에서 명사를 판단합니다. 22만개 문장으로부터 단어를 추출하고 명사를 판단하는 학습시간은 i7-5820 기준 39초가 걸렸습니다. 

In [7]:
%%time
nouns = noun_extractor.train_extract(
    sentences,
    min_noun_score=0.3,
    min_noun_frequency=20
)
print('num nouns = %d' % len(nouns))

scanning completed
(L,R) has (52547, 26347) tokens
building lr-graph completednum nouns = 14631
CPU times: user 30.1 s, sys: 124 ms, total: 30.2 s
Wall time: 30.1 s


nouns 는 dict[str] = namedtuple 형식으로 return 됩니다. namedtuple 인 NounScore 에는 어절의 왼쪽에 등장한 횟수, 명사 점수, R set 이 알려진 feature 인 비율이 저장되어 있습니다. 

In [8]:
nouns['뉴스']

NounScore(frequency=4814, score=0.43977009340659345, known_r_ratio=0.025966614352974746)

다음의 단어들이 명사로 추출되었는지 살펴보겠습니다. 

In [9]:
words = ['박근혜', '우병우', '민정수석', '트와이스', '아이오아이']
for word in words:
    print('%s is noun? %r' % (word, word in nouns))

박근혜 is noun? True
우병우 is noun? True
민정수석 is noun? True
트와이스 is noun? False
아이오아이 is noun? True


is_noun을 이용하면 각각의 단어에 대하여 명사 여부를 판단할 수도 있습니다. 

In [10]:
noun_extractor.is_noun('최순실', min_noun_score=0.3)

True

몇 개의 단어들을 입력하여 명사점수를 추출할 수도 있습니다. 

In [11]:
words = ['박근혜', '우병우', '민정수석', 
         '트와이스', '아이오아이', '최순실',
         '최순실게이트', '게이트', '콘서트']

for word in words:
    noun_score, known_r_ratio = noun_extractor.predict(word)
    print('%s : score=%.3f' % (word, noun_score))

박근혜 : score=0.538
우병우 : score=0.757
민정수석 : score=0.834
트와이스 : score=0.262
아이오아이 : score=0.980
최순실 : score=0.828
최순실게이트 : score=0.320
게이트 : score=0.745
콘서트 : score=0.769


(빈도수, 빈도수 * 명사점수) 두 가지 기준으로 상위 100개씩 선택하여 프린트를 하겠습니다. 

In [12]:
top100 = sorted(nouns.items(), 
    key=lambda x:-x[1].frequency)[:100]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.score), end='')


    기자 (0.67)    무단 (1.00)   20일 (0.91)   재배포 (1.00)    금지 (0.99)
    사진 (0.40)    지난 (0.99)  2016 (0.81)   뉴시스 (1.00)    함께 (0.97)
    대표 (0.39)    진행 (0.92)    오후 (0.99)  저작권자 (1.00)    때문 (1.00)
    이번 (0.99)    미국 (0.77)    이날 (0.69)   뉴스1 (0.94)    관련 (0.96)
    참석 (0.98)    제공 (0.96)   대통령 (0.49)   19일 (0.88)   이라고 (0.99)
    방송 (0.80)    오전 (1.00)    우리 (0.53)    경찰 (0.32)    영상 (0.54)
    사람 (0.52)    뉴스 (0.44)    공감 (0.69)    최근 (0.98)  연합뉴스 (0.69)
    모습 (0.73)    공개 (0.98)    하고 (0.80)    사랑 (0.91)    시작 (0.96)
    같은 (1.00)    올해 (0.39)    가장 (0.97)    이후 (0.98)    한국 (0.78)
    경우 (0.66)    라고 (1.00)    문제 (0.68)    이상 (0.86)    정부 (0.57)
    제보 (1.00)    하지 (1.00)    현재 (0.65)    북한 (0.80)    영화 (0.49)
    자신 (0.90)   예정이 (0.34)    국내 (0.97)    다양 (1.00)    시장 (0.67)
    중국 (0.70)    생각 (0.81)    상황 (0.83)    지역 (0.69)    기업 (0.82)
    세계 (0.98)    의혹 (0.83)    계획 (0.89)   관계자 (0.68)    사업 (0.51)
    확인 (0.95)    국회 (0.95)    조사 (0.93)    한편 (0.98)    지금 (0.54)
   있다고 (1

In [13]:
top100 = sorted(nouns.items(), 
    key=lambda x:-x[1].frequency * x[1].score)[:100]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.score), end='')


    무단 (1.00)   재배포 (1.00)    금지 (0.99)   20일 (0.91)    기자 (0.67)
    지난 (0.99)   뉴시스 (1.00)  2016 (0.81)    함께 (0.97)    오후 (0.99)
  저작권자 (1.00)    진행 (0.92)    때문 (1.00)    이번 (0.99)    관련 (0.96)
   뉴스1 (0.94)    참석 (0.98)   이라고 (0.99)    제공 (0.96)    오전 (1.00)
    사진 (0.40)    미국 (0.77)   19일 (0.88)    최근 (0.98)    공개 (0.98)
    같은 (1.00)    이날 (0.69)    방송 (0.80)    라고 (1.00)    시작 (0.96)
    이후 (0.98)    가장 (0.97)    제보 (1.00)    하지 (1.00)    사랑 (0.91)
    다양 (1.00)    국내 (0.97)    자신 (0.90)    이상 (0.86)    세계 (0.98)
    하고 (0.80)   있다고 (1.00)    한국 (0.78)    모습 (0.73)    한편 (0.98)
    북한 (0.80)    공감 (0.69)    국회 (0.95)    확인 (0.95)  연합뉴스 (0.69)
    당시 (0.99)    조사 (0.93)    설명 (0.95)    발표 (0.95)    계획 (0.89)
    상황 (0.83)    이에 (1.00)    생각 (0.81)    모두 (0.99)    기업 (0.82)
    대표 (0.39)    기록 (0.91)    의혹 (0.83)    필요 (0.97)    판단 (0.97)
    문제 (0.68)    경우 (0.66)    운영 (0.89)   대통령 (0.49)    우리 (0.53)
    없는 (1.00)    현재 (0.65)    이런 (1.00)    중국 (0.70)    이들 (0.92)
    가능 (0

## NewsNounExtractor

두번째의 명사 추출기는 NewsNounExtractor 입니다. 여러가지 후처리 과정을 추가하여, 뉴스데이터에서 좋은 성능을 낼 수 있도록 작업하는 중입니다. init 에 입력하는 arguments 는 동일합니다. 

In [14]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
/mnt/lovit/git/soynlp/soynlp


In [15]:
nouns = noun_extractor.train_extract(sentences)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (658116, 363342, 403882)
predicting noun score was done                                        4
before postprocessing 237871
_noun_scores_ 50196
checking hardrules ... done0 / 50196+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 34755
extracted 1994 compounds from eojeolss ... 87000 / 87714

In [16]:
words = ['박근혜', '우병우', '민정수석', 
         '트와이스', '아이오아이', '최순실',
         '최순실게이트', '게이트', '콘서트']

for word in words:
    if not word in nouns:
        continue
    score = nouns[word]
    print('%s: (score=%.3f, frequency=%d)' 
          % (word, score.score, score.frequency))

박근혜: (score=0.478, frequency=1507)
우병우: (score=0.757, frequency=721)
민정수석: (score=0.834, frequency=812)
아이오아이: (score=0.547, frequency=270)
최순실: (score=0.828, frequency=1878)
게이트: (score=0.745, frequency=307)
콘서트: (score=0.769, frequency=500)
